In [1]:
import pylab
import numpy as np 
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import csv
from bs4 import BeautifulSoup  
from wordcloud import WordCloud,STOPWORDS

import nltk
#nltk.download()  # Download text data sets, including stop words
from nltk.corpus import stopwords # Import the stop word list

import operator
import re

print "initial imports complete"

/Applications/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


initial imports complete


In [2]:
from bokeh.io import output_notebook, show
output_notebook()

from bokeh.plotting import figure
#p = figure(plot_width=400, plot_height=400)
#p.square([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=[10, 15, 20, 25, 30], color="firebrick", alpha=0.6)
#p.line(x, y, line_width=2)
#show(p) # show the results

from bokeh.charts import Scatter
#p = Scatter(flowers, x='petal_length', y='petal_width', color='species', legend='top_left')
#show(p)

from bokeh.charts import Bar
#p = Bar(autompg, label='yr', values='mpg', agg='median', group='origin', 
#        title="Median MPG by YR, grouped by ORIGIN", legend='top_left', tools='crosshair')
#show(p)

print "bokeh imports complete"

Loading BokehJS ...

bokeh imports complete


In [3]:
rev_and_user = pd.read_pickle('rev_and_user')

In [4]:
reviews = rev_and_user

#reviews from users with no friends
reviewsNF = reviews[reviews['nfriends'] == 1]
starnumNF = len(reviewsNF)

#reviews from users with many ("L"OTS) friends
reviewsLF = reviews[reviews['nfriends'] > 1]
starnumLF = len(reviewsLF)

starnum = len(reviews)

print starnumNF, starnumLF, starnum

884599 1340613 2225213


In [5]:
#Let's count the stars
starsNF = np.array([0.,0.,0.,0.,0.])  #NO friends
starsLF = np.array([0.,0.,0.,0.,0.])  #SOME review
stars = np.array([0.,0.,0.,0.,0.])  #ALL reviews

for i in range(0,5):
    starsNF[i] = float(len(reviewsNF[reviewsNF.stars == i+1])) / float(starnumNF) * 100.
    starsLF[i] = float(len(reviewsLF[reviewsLF.stars == i+1])) / float(starnumLF) * 100.
    stars[i] = float(len(reviews[reviews.stars == i+1])) / float(starnum) * 100.
    
print(starsNF)
print(starsLF)
print(stars)

[ 17.62425687   8.81529371   8.49164424  18.96203817  46.10676702]
[  7.80150573   8.35945944  15.44054847  31.6183716   36.78011477]
[ 11.70638496   8.54066555  12.67811216  26.58702785  40.48780948]


In [6]:
#Let's chart the stars
import matplotlib.patches as mpatches

multiple_bars = plt.figure()

x = np.array([1,2,3,4,5])

ax = plt.subplot(111)
ax.bar(x-0.2, starsNF,width=0.2,color='b',align='center', label='Zero Yelp friends')
ax.bar(x, stars,width=0.2,color='k',align='center', label='All reviews')
ax.bar(x+0.2,     starsLF,width=0.2,color='r',align='center', label='One or more Yelp friends')

ax.set_title('Distribution including all reviews')
ax.set_ylabel('Percentage of Reviewers')
ax.set_xlabel('Star Rating out of 5')

blue_patch =mpatches.Patch(color='blue', label='Zero Yelp Friends (40% of users)')
black_patch = mpatches.Patch(color='black', label='All reviews')
red_patch = mpatches.Patch(color='red', label='At least one Yelp friend (60% of users)')
ax.legend(handles=[blue_patch, black_patch, red_patch], loc=2)

plt.show()
plt.savefig('barchart_friends.png')
plt.clf()
plt.cla()
plt.close()

In [7]:
#reviews from FREQUENT users
reviewsFU = reviews[reviews['review_count'] >= 10]
starnumFU = len(reviewsFU)

#reviews from RARE users
reviewsRU = reviews[reviews['review_count'] < 10]
starnumRU = len(reviewsRU)

print starnumFU, starnumRU

1588758 636455


In [8]:
starsFU = np.array([0.,0.,0.,0.,0.])  #NO friends
starsRU = np.array([0.,0.,0.,0.,0.])  #SOME review

for i in range(0,5):
    starsFU[i] = float(len(reviewsFU[reviewsFU.stars == i+1])) / float(starnumFU) * 100.
    starsRU[i] = float(len(reviewsRU[reviewsRU.stars == i+1])) / float(starnumRU) * 100.
    
print(starsFU)
print(starsRU)
print(stars)

[  8.49046866   9.03661854  15.53389503  31.42624616  35.51277161]
[ 19.73415245   7.30263726   5.54933185  14.50707434  52.90680409]
[ 11.70638496   8.54066555  12.67811216  26.58702785  40.48780948]


In [10]:
multiple_bars = plt.figure()

x = np.array([1,2,3,4,5])

ax = plt.subplot(111)
ax.bar(x-0.2, starsRU,width=0.2,color='b',align='center', label='Rare users')
ax.bar(x, stars,width=0.2,color='k',align='center', label='All reviews')
ax.bar(x+0.2,     starsFU,width=0.2,color='r',align='center', label='Frequent users')

ax.set_title('Distribution including all reviews')
ax.set_ylabel('Percentage of Reviewers')
ax.set_xlabel('Star Rating out of 5')

blue_patch =mpatches.Patch(color='blue', label='Infrequent users \n (< 10 reviews, 29% of users)')
black_patch = mpatches.Patch(color='black', label='All reviews')
red_patch = mpatches.Patch(color='red', label='Frequent users \n (>= 10 reviews, 71% of users)')
ax.legend(handles=[blue_patch, black_patch, red_patch], loc=2)

plt.show()
plt.savefig('barchart_frequency.png')
plt.clf()
plt.cla()
plt.close()

In [11]:
#for the *users*, what's their ratio of 'cool' votes to 'useful' votes?
reviews['cool_to_useful'] = reviews['votes.cool_y'] / reviews['votes.useful_y']

#which users made return reviews?

return_users = reviews[reviews['return_visit'] == True]
print return_users.shape

(69792, 41)


In [12]:
return_users.head(3)
return_users2 = return_users.drop_duplicates('user_id')
print return_users2.shape

(32613, 41)


In [13]:
return_users2.head(10)

,user_id,text,votes.cool_x,business_id,votes.funny_x,stars,date,type_x,votes.useful_x,return_visit,...,compliments.list,votes.funny_y,compliments.photos,compliments.funny,votes.useful_y,friends2,nfriends,elite2,nelite,cool_to_useful
22,I_47G-R2_egp7ME5u_ltew,Came back tonight after a hiatus and missing o...,0,1qCuOcks5HRv67OHovAVpg,0,3,2015-02-07,review,0,True,...,NaN,8,NaN,NaN,46,"[['DsN2dJ6SnTEACr1qC5zpGg', 'UD7Y1CqfY6mDmRwI...",6,[[]],1,0.217391
117,JPPhyFE-UE453zA6K0TVgw,Stopped last night before a show at the O'Reil...,0,r9rub_ajcb-3yYsYYD5apw,0,5,2015-10-22,review,0,True,...,NaN,36,NaN,5.0,136,"[['fwsJGulnozT2U6FefsLiFw', 'Dsh4UCo9ny0XCrEJ...",31,"[[2014, 2015]]",2,0.514706
150,fhNxoMwwTipzjO8A9LFe8Q,Still sticking with my original rating. Just ...,1,sxeuOMBRllOHXJTaUlI9kQ,1,3,2012-12-15,review,3,True,...,NaN,75,1.0,2.0,220,"[['-QjeoLnIeKStNfGIq5KGDw', '_e5drmTJBSV0yiFB...",25,"[[2012, 2013, 2014]]",3,0.386364
296,l_szjd-ken3ma6oHDkTYXg,"Senti: A good, but forgettable meal.\n\nA frie...",0,nTPnejWVdbNkXRaPCJVDAQ,0,3,2015-07-06,review,1,True,...,NaN,49,2.0,4.0,202,"[['D3JoaaL9kEgnIs64SUOR2w', 'FP79xn8H2XXnMksR...",55,[[2015]],1,0.698020
311,4H-IfBDP5GkfNsQeHi4sHA,I really wanted to enjoy this place. I really ...,0,m3-Xe0exMPEWvY0U665f6w,0,1,2015-12-07,review,0,True,...,NaN,3,NaN,NaN,6,[['O49TQdh9E3BGeabd5m-Lbw']],1,[[]],1,0.500000
319,Qs5dcst13DnQJwaBcR4VGg,"Chilling with Kimberly Vasil, Mikey & Big Bob ...",0,sbW8qHJgzEIH42B0S-3New,0,4,2015-02-20,review,0,True,...,NaN,57,NaN,1.0,155,"[['9OZH1Ecw-qUkCW5MS0NefA', '9cCTmiJ7hz35rHId...",14,"[[2014, 2015]]",2,0.458065
378,8fApIAMHn2MZJFUiCQto5Q,"After further investigation, I opt for the but...",0,DzOPHyrG2XuVqKEvHCnN7w,0,5,2013-04-13,review,1,True,...,NaN,16,NaN,1.0,42,"[['UD7Y1CqfY6mDmRwIuCf6nA', 'tAKjY3bQXH51msJb...",2,[[]],1,0.238095
438,tAKjY3bQXH51msJbOHYPmQ,"Well, as stated in the previous review of thei...",0,YTqSkgOiiI9HG3hNH7ovfg,0,2,2012-09-01,review,2,True,...,NaN,195,NaN,3.0,457,"[['UFr-WCtnNbX8O3sI7h4qeQ', 'B5WkNWDxZ-baWoQc...",31,"[[2012, 2013, 2014, 2015]]",4,0.354486
591,JbWnXwjLaO1kncUkQOMajg,I bought sandwiches from here for all my co-wo...,0,Nqw610-DAmucQvlHZvrMkg,0,5,2011-07-27,review,1,True,...,NaN,163,NaN,6.0,208,"[['9a4W-3anHjCB28F7fAG0eg', 'DrKQzBFAvxhyjLgb...",23,[[]],1,0.375000
648,4-3IU5uUH90m21TWbZhhnA,I will say for the record that my first review...,0,Q7_NVo2y9GAyfBQ0LVNHbA,0,3,2013-03-20,review,0,True,...,NaN,27,1.0,1.0,97,"[['WEppeczBa630qanow0uqig', 'u1KESST6ATusVrn7...",9,[[]],1,0.226804


In [14]:
users = reviews.drop_duplicates('user_id')

users_plot = users[users['cool_to_useful'] > 0]

p = figure(plot_width=400, plot_height=400)
p.square(users_plot['cool_to_useful'], users_plot['average_stars'])
#p.line(x, y, line_width=2)
#show(p) # show the results

In [15]:
return_users = return_users2

users_plot = return_users[return_users['cool_to_useful'] > 0]

p = figure(plot_width=400, plot_height=400)
p.square(users_plot['cool_to_useful'], users_plot['average_stars'])
#p.line(x, y, line_width=2)
#show(p) # show the results

In [16]:
#reviews from "cool" users (higher cool_to_userful ratio)
reviewsVC = reviews[reviews['cool_to_useful'] > 1]
starnumVC = len(reviewsVC)

#reviews from not-cool users
reviewsNC = reviews[(reviews['cool_to_useful'] < 1) & (reviews['cool_to_useful'] > 0)]
starnumNC = len(reviewsNC)

print starnumVC, starnumNC

39337 1737263


In [17]:
starsVC = np.array([0.,0.,0.,0.,0.])  #NO friends
starsNC = np.array([0.,0.,0.,0.,0.])  #SOME review

for i in range(0,5):
    starsVC[i] = float(len(reviewsVC[reviewsVC.stars == i+1])) / float(starnumVC) * 100.
    starsNC[i] = float(len(reviewsNC[reviewsNC.stars == i+1])) / float(starnumNC) * 100.
    
print(starsVC)
print(starsNC)
print(stars)

[  7.53743295   4.86310598   7.49675878  22.06573963  58.03696266]
[  9.66526082   8.88811884  14.49688389  29.66931317  37.28042329]
[ 11.70638496   8.54066555  12.67811216  26.58702785  40.48780948]


In [18]:
multiple_bars = plt.figure()

x = np.array([1,2,3,4,5])

ax = plt.subplot(111)
ax.bar(x-0.2, starsVC,width=0.2,color='b',align='center', label='Cool-rated users')
ax.bar(x, stars,width=0.2,color='k',align='center', label='All reviews')
ax.bar(x+0.2,     starsNC,width=0.2,color='r',align='center', label='Not-cool-rates users')

ax.set_title('Distribution including all reviews')
ax.set_ylabel('Percentage of Reviewers')
ax.set_xlabel('Star Rating out of 5')

blue_patch =mpatches.Patch(color='blue', label='Cool-rated users (2.5% of users)')
black_patch = mpatches.Patch(color='black', label='All reviews')
red_patch = mpatches.Patch(color='red', label='Not-cool-rated users (97.5% of users)')
ax.legend(handles=[blue_patch, black_patch, red_patch], loc=2)

plt.show()
plt.savefig('barchart_cool.png')
plt.clf()
plt.cla()
plt.close()

In [19]:
#reviews_plot = reviews[reviews['day'] % 10 == 0]
#print reviews_plot.shape

x = [2005, 2005, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015] #yr
y = [4.03043478261, 3.81019626616, 3.82179684384, 3.72128480039, \
     3.69752414544, 3.73993035403, 3.74245019382, 3.73217807719,  \
     3.748802882, 3.75978920904,  3.78084885141] #avg star-rating that year

p = figure(plot_width=400, plot_height=400)
#p.square(reviews_plot['year'], reviews_plot['stars'])
p.line(x, y, line_width=4)
p.xaxis.axis_label = "year"
p.yaxis.axis_label = "average star rating"
show(p) # show the results

In [20]:
from bokeh.models import FixedTicker
p = figure(plot_width=400, plot_height=400)
a = [1,2,3,4,5,6,7,8,9,10,11,12]
b = [3.7543713913,3.75327611039,3.74027241108,3.74996604491,3.74702416671,\
     3.76055435981,3.7739954756,3.76707679646,3.7589486608,3.76248644478,\
     3.75922892093,3.73848616722]

p.line(a, b, line_width=4)
p.xaxis.axis_label = "month"
p.yaxis.axis_label = "average star rating"
#p.yaxis[0].ticker=FixedTicker(ticks=[3.7, 3.8, 3.9, 4.0, 4.1])
show(p) # show the results

In [21]:
reviews_first_visit = reviews[reviews['first_visit'] == True] 
reviews_return_visit = reviews[reviews['return_visit'] == True] 

In [22]:
#reviews from users with no friends
reviewsNF = reviews_first_visit[reviews_first_visit['nfriends'] == 1]
starnumNF = len(reviewsNF)

#reviews from users with many ("L"OTS) friends
reviewsLF = reviews_first_visit[reviews_first_visit['nfriends'] > 1]
starnumLF = len(reviewsLF)

starnum = len(reviews_first_visit)

print starnumNF, starnumLF, starnum

#reviews from FREQUENT users
reviewsFU = reviews_first_visit[reviews_first_visit['review_count'] >= 10]
starnumFU = len(reviewsFU)

#reviews from RARE users
reviewsRU = reviews_first_visit[reviews_first_visit['review_count'] < 10]
starnumRU = len(reviewsRU)

print starnumFU, starnumRU

#reviews from "cool" users (higher cool_to_userful ratio)
reviewsVC = reviews_first_visit[reviews_first_visit['cool_to_useful'] > 1]
starnumVC = len(reviewsVC)

#reviews from not-cool users
reviewsNC = reviews_first_visit[(reviews_first_visit['cool_to_useful'] < 1) & (reviews_first_visit['cool_to_useful'] > 0)]
starnumNC = len(reviewsNC)

print starnumVC, starnumNC


21208 48584 69792
57582 12210
868 61154


In [23]:
starnum = len(reviews)

#Let's count the stars FOR FRIENDS
starsNF = np.array([0.,0.,0.,0.,0.])  #NO friends
starsLF = np.array([0.,0.,0.,0.,0.])  #SOME review
stars = np.array([0.,0.,0.,0.,0.])  #ALL reviews

for i in range(0,5):
    starsNF[i] = float(len(reviewsNF[reviewsNF.stars == i+1])) / float(starnumNF) * 100.
    starsLF[i] = float(len(reviewsLF[reviewsLF.stars == i+1])) / float(starnumLF) * 100.
    stars[i] = float(len(reviews[reviews.stars == i+1])) / float(starnum) * 100.
    
print(starsNF)
print(starsLF)
print(stars)

#Let's chart the stars
import matplotlib.patches as mpatches

multiple_bars = plt.figure()

x = np.array([1,2,3,4,5])

ax = plt.subplot(111)
ax.bar(x-0.2, starsNF,width=0.2,color='b',align='center', label='Zero Yelp friends')
ax.bar(x, stars,width=0.2,color='k',align='center', label='All reviews')
ax.bar(x+0.2,     starsLF,width=0.2,color='r',align='center', label='One or more Yelp friends')

ax.set_title('Initial reviews only')
ax.set_ylabel('Percentage of Reviewers')
ax.set_xlabel('Star Rating out of 5')

blue_patch =mpatches.Patch(color='blue', label='Zero Yelp Friends (30%)')
black_patch = mpatches.Patch(color='black', label='All reviews')
red_patch = mpatches.Patch(color='red', label='At least one Yelp friend (70%)')
ax.legend(handles=[blue_patch, black_patch, red_patch], loc=2)

plt.show()
plt.savefig('barchart_friendsFR.png')
plt.clf()
plt.cla()
plt.close()

#############################################################
##################################################
#Let's count the stars FOR FREQUENT USERS
starsFU = np.array([0.,0.,0.,0.,0.])  #NO friends
starsRU = np.array([0.,0.,0.,0.,0.])  #SOME review

for i in range(0,5):
    starsFU[i] = float(len(reviewsFU[reviewsFU.stars == i+1])) / float(starnumFU) * 100.
    starsRU[i] = float(len(reviewsRU[reviewsRU.stars == i+1])) / float(starnumRU) * 100.
    
print(starsFU)
print(starsRU)
print(stars)

#Let's chart the stars
import matplotlib.patches as mpatches

multiple_bars = plt.figure()

x = np.array([1,2,3,4,5])

ax = plt.subplot(111)
ax.bar(x-0.2, starsRU,width=0.2,color='b',align='center', label='Infrequent users')
ax.bar(x, stars,width=0.2,color='k',align='center', label='All reviews')
ax.bar(x+0.2,starsFU,width=0.2,color='r',align='center', label='Frequent users')

ax.set_title('Initial reviews only')
ax.set_ylabel('Percentage of Reviewers')
ax.set_xlabel('Star Rating out of 5')

blue_patch =mpatches.Patch(color='blue', label='Infrequent users (17%)')
black_patch = mpatches.Patch(color='black', label='All reviews')
red_patch = mpatches.Patch(color='red', label='Frequent users (83%)')
ax.legend(handles=[blue_patch, black_patch, red_patch], loc=2)

plt.show()
plt.savefig('barchart_frequencyFR.png')
plt.clf()
plt.cla()
plt.close()
       
##########################################################
#############################################################       
#Let's count the stars FOR FREQUENT USERS
starsVC = np.array([0.,0.,0.,0.,0.])  #NO friends
starsNC = np.array([0.,0.,0.,0.,0.])  #SOME review

for i in range(0,5):
    starsVC[i] = float(len(reviewsVC[reviewsVC.stars == i+1])) / float(starnumVC) * 100.
    starsNC[i] = float(len(reviewsNC[reviewsNC.stars == i+1])) / float(starnumNC) * 100.
    
print(starsVC)
print(starsNC)

#Let's chart the stars
import matplotlib.patches as mpatches

multiple_bars = plt.figure()

x = np.array([1,2,3,4,5])

ax = plt.subplot(111)
ax.bar(x-0.2, starsVC,width=0.2,color='b',align='center', label='Cool-rated users')
ax.bar(x, stars,width=0.2,color='k',align='center', label='All reviews')
ax.bar(x+0.2,     starsNC,width=0.2,color='r',align='center', label='Not-ool-rated users')

ax.set_title('Initial reviews only')
ax.set_ylabel('Percentage of Reviewers')
ax.set_xlabel('Star Rating out of 5')

blue_patch =mpatches.Patch(color='blue', label='Cool-rated users (1.2%)')
black_patch = mpatches.Patch(color='black', label='All reviews')
red_patch = mpatches.Patch(color='red', label='Not-cool-rated users (98.8%)')
ax.legend(handles=[blue_patch, black_patch, red_patch], loc=2)

plt.show()
plt.savefig('barchart_coolFR.png')
plt.clf()
plt.cla()
plt.close()

[ 21.03451528   8.87400981   8.15258393  16.43247831  45.50641267]
[  8.14259839   7.98411     14.25778034  30.88876997  38.72674131]
[ 11.70638496   8.54066555  12.67811216  26.58702785  40.48780948]
[  9.24941822   8.47834393  14.03216283  30.13615366  38.10392136]
[ 25.31531532   7.1990172    4.71744472   9.32841933  53.43980344]
[ 11.70638496   8.54066555  12.67811216  26.58702785  40.48780948]
[  8.75576037   4.83870968   4.95391705  15.20737327  66.24423963]
[ 10.44903032   8.45243157  13.38751349  28.66860712  39.0424175 ]


In [24]:
#reviews from users with no friends
reviewsNF = reviews_return_visit[reviews_return_visit['nfriends'] == 1]
starnumNF = len(reviewsNF)

#reviews from users with many ("L"OTS) friends
reviewsLF = reviews_return_visit[reviews_return_visit['nfriends'] > 1]
starnumLF = len(reviewsLF)

starnum = len(reviews_first_visit)

print starnumNF, starnumLF, starnum

#reviews from FREQUENT users
reviewsFU = reviews_return_visit[reviews_return_visit['review_count'] >= 10]
starnumFU = len(reviewsFU)

#reviews from RARE users
reviewsRU = reviews_return_visit[reviews_return_visit['review_count'] < 10]
starnumRU = len(reviewsRU)

print starnumFU, starnumRU

#reviews from "cool" users (higher cool_to_userful ratio)
reviewsVC = reviews_return_visit[reviews_return_visit['cool_to_useful'] > 1]
starnumVC = len(reviewsVC)

#reviews from not-cool users
reviewsNC = reviews_return_visit[(reviews_return_visit['cool_to_useful'] < 1) & (reviews_return_visit['cool_to_useful'] > 0)]
starnumNC = len(reviewsNC)

print starnumVC, starnumNC

21208 48584 69792
57582 12210
868 61154


In [25]:
starnum = len(reviews)

#Let's count the stars FOR FRIENDS
starsNF = np.array([0.,0.,0.,0.,0.])  #NO friends
starsLF = np.array([0.,0.,0.,0.,0.])  #SOME review
stars = np.array([0.,0.,0.,0.,0.])  #ALL reviews

for i in range(0,5):
    starsNF[i] = float(len(reviewsNF[reviewsNF.stars == i+1])) / float(starnumNF) * 100.
    starsLF[i] = float(len(reviewsLF[reviewsLF.stars == i+1])) / float(starnumLF) * 100.
    stars[i] = float(len(reviews[reviews.stars == i+1])) / float(starnum) * 100.
    
print(starsNF)
print(starsLF)
print(stars)

#Let's chart the stars
import matplotlib.patches as mpatches

multiple_bars = plt.figure()

x = np.array([1,2,3,4,5])

ax = plt.subplot(111)
ax.bar(x-0.2, starsNF,width=0.2,color='b',align='center', label='Zero Yelp friends')
ax.bar(x, stars,width=0.2,color='k',align='center', label='All reviews')
ax.bar(x+0.2,     starsLF,width=0.2,color='r',align='center', label='One or more Yelp friends')

ax.set_title('Return reviews only')
ax.set_ylabel('Percentage of Reviewers')
ax.set_xlabel('Star Rating out of 5')

blue_patch =mpatches.Patch(color='blue', label='Zero Yelp Friends (30%)')
black_patch = mpatches.Patch(color='black', label='All reviews')
red_patch = mpatches.Patch(color='red', label='At least one Yelp friend (70%)')
ax.legend(handles=[blue_patch, black_patch, red_patch], loc=2)

plt.show()
plt.savefig('barchart_friendsRR.png')
plt.clf()
plt.cla()
plt.close()

#############################################################
##################################################
#Let's count the stars FOR FREQUENT USERS
starsFU = np.array([0.,0.,0.,0.,0.])  #NO friends
starsRU = np.array([0.,0.,0.,0.,0.])  #SOME review

for i in range(0,5):
    starsFU[i] = float(len(reviewsFU[reviewsFU.stars == i+1])) / float(starnumFU) * 100.
    starsRU[i] = float(len(reviewsRU[reviewsRU.stars == i+1])) / float(starnumRU) * 100.
    
print(starsFU)
print(starsRU)
print(stars)

#Let's chart the stars
import matplotlib.patches as mpatches

multiple_bars = plt.figure()

x = np.array([1,2,3,4,5])

ax = plt.subplot(111)
ax.bar(x-0.2, starsRU,width=0.2,color='b',align='center', label='Infrequent users')
ax.bar(x, stars,width=0.2,color='k',align='center', label='All reviews')
ax.bar(x+0.2,starsFU,width=0.2,color='r',align='center', label='Frequent users')

ax.set_title('Return reviews only')
ax.set_ylabel('Percentage of Reviewers')
ax.set_xlabel('Star Rating out of 5')

blue_patch =mpatches.Patch(color='blue', label='Infrequent users (17%)')
black_patch = mpatches.Patch(color='black', label='All reviews')
red_patch = mpatches.Patch(color='red', label='Frequent users (83%)')
ax.legend(handles=[blue_patch, black_patch, red_patch], loc=2)

plt.show()
plt.savefig('barchart_frequencyRR.png')
plt.clf()
plt.cla()
plt.close()
       
##########################################################
#############################################################       
#Let's count the stars FOR FREQUENT USERS
starsVC = np.array([0.,0.,0.,0.,0.])  #NO friends
starsNC = np.array([0.,0.,0.,0.,0.])  #SOME review

for i in range(0,5):
    starsVC[i] = float(len(reviewsVC[reviewsVC.stars == i+1])) / float(starnumVC) * 100.
    starsNC[i] = float(len(reviewsNC[reviewsNC.stars == i+1])) / float(starnumNC) * 100.
    
print(starsVC)
print(starsNC)

#Let's chart the stars
import matplotlib.patches as mpatches

multiple_bars = plt.figure()

x = np.array([1,2,3,4,5])

ax = plt.subplot(111)
ax.bar(x-0.2, starsVC,width=0.2,color='b',align='center', label='Cool-rated users')
ax.bar(x, stars,width=0.2,color='k',align='center', label='All reviews')
ax.bar(x+0.2,     starsNC,width=0.2,color='r',align='center', label='Not-ool-rated users')

ax.set_title('Return reviews only')
ax.set_ylabel('Percentage of Reviewers')
ax.set_xlabel('Star Rating out of 5')

blue_patch =mpatches.Patch(color='blue', label='Cool-rated users (1.2%)')
black_patch = mpatches.Patch(color='black', label='All reviews')
red_patch = mpatches.Patch(color='red', label='Not-cool-rated users (98.8%)')
ax.legend(handles=[blue_patch, black_patch, red_patch], loc=2)

plt.show()
plt.savefig('barchart_coolRR.png')
plt.clf()
plt.cla()
plt.close()

[ 22.11901169   8.09600151   8.36005281  14.77744247  46.64749151]
[ 11.42968879   9.06471266  13.11131237  24.56775893  41.82652725]
[ 11.70638496   8.54066555  12.67811216  26.58702785  40.48780948]
[ 12.85297489   9.51512625  13.00406377  23.9866625   40.64117259]
[ 23.28419328   5.25798526   5.36445536  10.3030303   55.79033579]
[ 11.70638496   8.54066555  12.67811216  26.58702785  40.48780948]
[  8.98617512   4.14746544   4.83870968  14.51612903  67.51152074]
[ 13.71782712   9.23079439  12.4995912   23.01239494  41.53939235]
